In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun  3 19:23:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install llm-rs --quiet
!pip install langchain --quiet
!pip install llm-rs[langchain] --quiet
!cp -r /kaggle/input/peft-main/src/peft .

cp: cannot stat '/kaggle/input/peft-main/src/peft': No such file or directory


In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import torch
from torch import nn
import numpy as np
import re
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, LlamaForSequenceClassification
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
from tqdm import tqdm

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

In [5]:
model_path = 'openlm-research/open_llama_3b' # https://huggingface.co/openlm-research/open_llama_3b
num_labels = 4 # 'Positive', 'Neutral', 'Negative', 'Irrelevant' (https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2)

model = LlamaForSequenceClassification.from_pretrained(model_path, num_labels=num_labels, torch_dtype=torch.float16, device_map='cuda:0')
# Tokenize the input: Use the AutoTokenizer class from the transformers library to tokenize your input text. Set the tokenizer options according to your classification task.

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=False,
    trust_remote_code=True,
    padding_side="left",
    pad_token="<|endoftext|>"
)

model.resize_token_embeddings(len(tokenizer)) # https://github.com/huggingface/transformers/issues/1805

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at openlm-research/open_llama_3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Embedding(32001, 3200)

In [6]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"], # https://github.com/huggingface/peft/blob/632997d1fb776c3cf05d8c2537ac9a98a7ce9435/src/peft/utils/other.py#L202
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)

In [19]:
twitterDFTrain = pd.read_csv("twitter_training.csv")
twitterDFVal = pd.read_csv("twitter_validation.csv")

In [20]:
twitterDFTrain


,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


from matplotlib import pyplot as plt
_df_0['2401'].plot(kind='hist', bins=20, title='2401')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['sentiment'].plot(kind='hist', bins=20, title='sentiment')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Positive').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='2401', y='sentiment', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_4['2401'].plot(kind='line', figsize=(8, 4), title='2401')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_5['sentiment'].plot(kind='line', figsize=(8, 4), title='sentiment')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_6['Positive'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_6, x='2401', y='Positive', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['Positive'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='sentiment', y='Positive', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [21]:
twitterClasses = {'Negative':0, 'Positive':1, 'Neutral':2, 'Irrelevant':3}
twitterDFTrain["sentiment"] = 0
twitterDFVal["sentiment"] = 0
for keyStr in twitterClasses.keys():
    twitterDFTrain["sentiment"][twitterDFTrain["Positive"]==keyStr] = twitterClasses[keyStr]
    twitterDFVal["sentiment"][twitterDFVal["Irrelevant"]==keyStr] = twitterClasses[keyStr]

twitterDFTrain["text"] = twitterDFTrain[list(twitterDFTrain.columns)[3]]
twitterDFVal["text"] = twitterDFVal[list(twitterDFVal.columns)[3]]
twitterDFTrain = twitterDFTrain[["text", "sentiment"]] # We only need the tweets and their sentiments for training LLaMA
twitterDFVal = twitterDFVal[["text", "sentiment"]] # We only need the tweets and their sentiments for training LLaMA

In [22]:
def basic_cleaning(text):
    # Capture swear words that are **** out return text
    text=re.sub(r'https?://www\.\S+\.com','',text)
    text=re.sub(r'[^A-Za-z|\s]','',text)
    text=re.sub(r'\*+','swear',text)
    return text

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F" #emoticons
                               u"\U0001F300-\U0001F5FF" #symbols & pictographs
                               u"\U0001F680-\U0001F6FF" #transport & map symbols
                               u"\U0001F1E0-\U0001F1FF" #flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_multiplechars(text):
    text = re.sub(r'(.)\1{3,}',r'\1', text)
    return text

def clean(df):
    for col in ['text']:#,'selected_text']:
        df[col]=df[col].astype(str).apply(lambda x:basic_cleaning(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_emoji(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_html(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_multiplechars(x))
    return df

In [23]:
twitterDFTrain_clean = clean(twitterDFTrain)
twitterDFVal_clean = clean(twitterDFVal)

In [24]:
twitterDFTrain_clean # train set contents


,text,sentiment
0,I am coming to the borders and I will kill you...,1
1,im getting on borderlands and i will kill you all,1
2,im coming on borderlands and i will murder you...,1
3,im getting on borderlands and i will murder y...,1
4,im getting into borderlands and i can murder y...,1
...,...,...
74676,Just realized that the Windows partition of my...,1
74677,Just realized that my Mac window partition is ...,1
74678,Just realized the windows partition of my Mac ...,1
74679,Just realized between the windows partition of...,1


In [25]:
twitterDFVal_clean # validation set contents


,text,sentiment
0,BBC News Amazon boss Jeff Bezos rejects claim...,2
1,Microsoft Why do I pay for WORD when it functi...,0
2,CSGO matchmaking is so full of closet hacking ...,0
3,Now the President is slapping Americans in the...,2
4,Hi EAHelp Ive had Madeleine McCann in my cella...,0
...,...,...
994,Toronto is the arts and culture capital of Ca...,3
995,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...,3
996,Today sucked so its time to drink wine n play ...,1
997,Bought a fraction of Microsoft today Small wins,1


In [26]:
class CustomDataset(Dataset):
    def __init__(self, twitterDF):
        self.twitterDF = twitterDF

    def __len__(self):
        return len(self.twitterDF.index)

    def __getitem__(self, idx):
        return np.array([idx])

In [27]:
training_data = CustomDataset(twitterDFTrain_clean)
validation_data = CustomDataset(twitterDFVal_clean)

train_dataloader = DataLoader(training_data, batch_size=4, shuffle=True)
val_dataloader = DataLoader(validation_data, batch_size=4, shuffle=False)


In [28]:
epochs = 1
# Let's just train on a small batch of the dataset and see the validation score
earlyStopAtTrainBatchIdx = 10000 # Set these variables to None to train and validate on all the dataset
earlyStopAtValBatchIdx = 200 # Set these variables to None to train and validate on all the dataset
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print("epochs: ", epochs)
    for batchIdx, sampledIdx in enumerate(tqdm(train_dataloader, position=0, leave=True)):
        sampledIdx = sampledIdx.cpu().data.numpy()
        model.train()
        optimizer.zero_grad()

        sampledRowText = list(twitterDFTrain_clean["text"].iloc[list(sampledIdx.flatten())])
        sampledRowLabels = torch.tensor(list(twitterDFTrain_clean["sentiment"].iloc[list(sampledIdx.flatten())])).to("cuda")
        encoded_input = tokenizer(sampledRowText, truncation=True, padding=True, return_tensors='pt').to("cuda") # Output shape: [bs, num_Labels]
        encoded_inputIds = encoded_input["input_ids"].to("cuda")
        encoded_attnMask = encoded_input["attention_mask"].to("cuda")
        outputs = model(input_ids=encoded_inputIds, attention_mask=encoded_attnMask)
        logits = outputs.logits
        loss = loss_fn(logits.squeeze(), sampledRowLabels)
        loss.backward()
        optimizer.step()

        if earlyStopAtTrainBatchIdx!=None:
            if batchIdx==earlyStopAtTrainBatchIdx: break
    # Validation
    predLs = []
    labelLs = []
    for batchIdx, sampledIdx in enumerate(val_dataloader):
        model.eval()

        sampledRowText = list(twitterDFVal_clean["text"].iloc[list(sampledIdx.flatten())])
        sampledRowLabels = torch.tensor(list(twitterDFVal_clean["sentiment"].iloc[list(sampledIdx.flatten())]))
        encoded_input = tokenizer(sampledRowText, truncation=True, padding=True, return_tensors='pt').to("cuda") # Output shape: [bs, num_Labels]
        encoded_inputIds = encoded_input["input_ids"].to("cuda")
        encoded_attnMask = encoded_input["attention_mask"].to("cuda")
        outputs = model(input_ids=encoded_inputIds, attention_mask=encoded_attnMask)
        logits = outputs.logits
        predLs.append(torch.argmax(logits, dim=1).flatten().cpu().data.numpy())
        labelLs.append(sampledRowLabels.cpu().data.numpy())

        if earlyStopAtValBatchIdx!=None:
            if batchIdx==earlyStopAtValBatchIdx: break
    predLs = torch.tensor(predLs).flatten()
    labelLs = torch.tensor(labelLs).flatten()
    accuracy = Accuracy(task="multiclass", num_classes=4)
    valAcc = float(accuracy(predLs, labelLs))
    print("twitter sentiment validation accuracy: ", valAcc)

epochs:  1


 54%|█████▎    | 10000/18671 [27:54<24:12,  5.97it/s]


twitter sentiment validation accuracy:  0.26865673065185547
